In [1]:
%cd /home/luke-padmore/Source/flow-matching-mnist
%load_ext autoreload
%autoreload 2
from models.config import UNetConfig
from models.unet import UNet

/home/luke-padmore/Source/flow-matching-mnist


## Testing it instantiates correctly

In [2]:
cfg = UNetConfig(
    channels=(1, 64, 128, 256),  
    d_trunk=32,
    d_concat=8,
    group_norm_size=8,
    d_time=128,
    max_time_period=10000.0,
    activation="silu",
    upsample_mode="nearest",
)

model = UNet.from_config(cfg)
print(model)

UNet(
  (encoder): Encoder(
    (initial_conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): ConvDownblock(
        (conv): Sequential(
          (0): GroupNorm(8, 72, eps=1e-05, affine=True)
          (1): Conv2d(72, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (2): SiLU()
          (3): GroupNorm(8, 64, eps=1e-05, affine=True)
          (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (5): SiLU()
        )
        (down): Sequential(
          (0): GroupNorm(8, 64, eps=1e-05, affine=True)
          (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (2): SiLU()
        )
        (time_emb_mlp): Linear(in_features=32, out_features=8, bias=True)
      )
      (1): ConvDownblock(
        (conv): Sequential(
          (0): GroupNorm(8, 72, eps=1e-05, affine=True)
          (1): Conv2d(72, 128, kernel_size=(3, 3)

## Testing forward pass

In [3]:
import torchvision
import torch
import matplotlib.pyplot as plt 
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from models.unet import UNet
import sys
from pathlib import Path 
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

batch_size = 64
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Pad(2,padding_mode='constant'),
    transforms.Normalize((0.5,), (0.5,))]
)
trainset = torchvision.datasets.MNIST(root = '/home/luke-padmore/Source/flow-matching-mnist/data',
                                      train=True,
                                      download=True,
                                      transform=transform)
trainloader = DataLoader(trainset,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=4)

testset = torchvision.datasets.MNIST(root = '/home/luke-padmore/Source/flow-matching-mnist/data',
                                      train=False,
                                      download=True,
                                      transform=transform)
testloader = DataLoader(trainset,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=4)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataiter = iter(trainloader)
images, labels = next(dataiter)

cfg = UNetConfig(
    channels=(1, 64, 128, 256),  
    d_trunk=32,
    d_concat=8,
    group_norm_size=8,
    d_time=128,
    max_time_period=10000.0,
    activation="silu",
    upsample_mode="nearest",
)

model = UNet.from_config(cfg).to(device)
images = images.to(device)
t = torch.rand((images.shape[0]),device=device).view(-1,1)
out = model(images,t)
print(out.shape)

torch.Size([64, 1, 32, 32])


Working :P